## 芸人情報の定量分析

全ユニット情報から各芸人情報を取得し、誕生日・年齢・出身地・事務所・芸歴を比較

https://www.geireki.net/lists/member

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
folder_name_raw = "./data_additional/raw"
folder_name_tidy = "./data_additional/tidy"
print(os.path.exists(folder_name_raw))
print(os.path.exists(folder_name_tidy))

True
True


In [3]:
# User-Agent の設定
dummy_user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'


In [4]:
# コンビ一覧のルートURL
combi_url_root = "https://www.geireki.net/lists/member/combi?disband=1"

# 124ページに分かれているのでリストを作成
combi_urls = [combi_url_root]
for i in range(123):
    combi_urls.append(combi_url_root + f"&page={i+2}")
combi_urls

['https://www.geireki.net/lists/member/combi?disband=1',
 'https://www.geireki.net/lists/member/combi?disband=1&page=2',
 'https://www.geireki.net/lists/member/combi?disband=1&page=3',
 'https://www.geireki.net/lists/member/combi?disband=1&page=4',
 'https://www.geireki.net/lists/member/combi?disband=1&page=5',
 'https://www.geireki.net/lists/member/combi?disband=1&page=6',
 'https://www.geireki.net/lists/member/combi?disband=1&page=7',
 'https://www.geireki.net/lists/member/combi?disband=1&page=8',
 'https://www.geireki.net/lists/member/combi?disband=1&page=9',
 'https://www.geireki.net/lists/member/combi?disband=1&page=10',
 'https://www.geireki.net/lists/member/combi?disband=1&page=11',
 'https://www.geireki.net/lists/member/combi?disband=1&page=12',
 'https://www.geireki.net/lists/member/combi?disband=1&page=13',
 'https://www.geireki.net/lists/member/combi?disband=1&page=14',
 'https://www.geireki.net/lists/member/combi?disband=1&page=15',
 'https://www.geireki.net/lists/member/co

In [8]:
# ピン一覧のルートURL
pin_url_root = "https://www.geireki.net/lists/member/pin?disband=1"

# 78ページに分かれているのでリストを作成
pin_urls = [pin_url_root]
for i in range(77):
    pin_urls.append(pin_url_root + f"&page={i+2}")
pin_urls

['https://www.geireki.net/lists/member/pin?disband=1',
 'https://www.geireki.net/lists/member/pin?disband=1&page=2',
 'https://www.geireki.net/lists/member/pin?disband=1&page=3',
 'https://www.geireki.net/lists/member/pin?disband=1&page=4',
 'https://www.geireki.net/lists/member/pin?disband=1&page=5',
 'https://www.geireki.net/lists/member/pin?disband=1&page=6',
 'https://www.geireki.net/lists/member/pin?disband=1&page=7',
 'https://www.geireki.net/lists/member/pin?disband=1&page=8',
 'https://www.geireki.net/lists/member/pin?disband=1&page=9',
 'https://www.geireki.net/lists/member/pin?disband=1&page=10',
 'https://www.geireki.net/lists/member/pin?disband=1&page=11',
 'https://www.geireki.net/lists/member/pin?disband=1&page=12',
 'https://www.geireki.net/lists/member/pin?disband=1&page=13',
 'https://www.geireki.net/lists/member/pin?disband=1&page=14',
 'https://www.geireki.net/lists/member/pin?disband=1&page=15',
 'https://www.geireki.net/lists/member/pin?disband=1&page=16',
 'https:

In [12]:
# トリオ一覧のルートURL
trio_url_root = "https://www.geireki.net/lists/member/trio?disband=1"

# 11ページに分かれているのでリストを作成
trio_urls = [trio_url_root]
for i in range(10):
    trio_urls.append(trio_url_root + f"&page={i+2}")
trio_urls

['https://www.geireki.net/lists/member/trio?disband=1',
 'https://www.geireki.net/lists/member/trio?disband=1&page=2',
 'https://www.geireki.net/lists/member/trio?disband=1&page=3',
 'https://www.geireki.net/lists/member/trio?disband=1&page=4',
 'https://www.geireki.net/lists/member/trio?disband=1&page=5',
 'https://www.geireki.net/lists/member/trio?disband=1&page=6',
 'https://www.geireki.net/lists/member/trio?disband=1&page=7',
 'https://www.geireki.net/lists/member/trio?disband=1&page=8',
 'https://www.geireki.net/lists/member/trio?disband=1&page=9',
 'https://www.geireki.net/lists/member/trio?disband=1&page=10',
 'https://www.geireki.net/lists/member/trio?disband=1&page=11']

In [10]:
# グループ一覧のルートURL（1ページのみ）
group_url_root = "https://www.geireki.net/lists/member/group?disband=1"



In [16]:
# コンビ詳細ページへのURLリストを作成

combi_page_urls = []

for combi_url in combi_urls:
    request = requests.get(combi_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    combi_pages = soup.select("#top > div > div > div > table > tbody > tr")
    for combi_page in combi_pages:
        combi_page_a = combi_page.select_one("td:nth-child(1) > a")
        url = combi_page_a.get("href")
        combi_name = combi_page_a.get_text(strip=True)
        combi_page_urls.append({"unit_name": combi_name, "page_url": url})
    time.sleep(1)

combi_page_df = pd.DataFrame(combi_page_urls)
combi_page_df

,unit_name,page_url
0,アームストロング,https://www.geireki.net/entertainers/1
1,アイスクリーム,https://www.geireki.net/entertainers/2
2,アイデンティティ,https://www.geireki.net/entertainers/3
3,アイナラット,https://www.geireki.net/entertainers/4
4,アイロンヘッド,https://www.geireki.net/entertainers/5
...,...,...
1846,セッツァー,https://www.geireki.net/entertainers/3186
1847,レモンコマドリ,https://www.geireki.net/entertainers/3188
1848,まぐろ兄弟,https://www.geireki.net/entertainers/3189
1849,幸せのとり,https://www.geireki.net/entertainers/3194


In [19]:
# ピン詳細ページへのURLリストを作成

pin_page_urls = []

for combi_url in pin_urls:
    request = requests.get(combi_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    combi_pages = soup.select("#top > div > div > div > table > tbody > tr")
    for combi_page in combi_pages:
        combi_page_a = combi_page.select_one("td:nth-child(1) > a")
        url = combi_page_a.get("href")
        combi_name = combi_page_a.get_text(strip=True)
        pin_page_urls.append({"unit_name": combi_name, "page_url": url})
    time.sleep(1)

pin_page_df = pd.DataFrame(pin_page_urls)
pin_page_df

,unit_name,page_url
0,あぁ～しらき,https://www.geireki.net/entertainers/1847
1,R藤本,https://www.geireki.net/entertainers/1848
2,あいすけ,https://www.geireki.net/entertainers/1849
3,青木さやか,https://www.geireki.net/entertainers/1850
4,青木隆治,https://www.geireki.net/entertainers/1851
...,...,...
1162,リクロジー,https://www.geireki.net/entertainers/3185
1163,もじゃ,https://www.geireki.net/entertainers/3187
1164,マッスルオペラちゃん,https://www.geireki.net/entertainers/3190
1165,福はら777,https://www.geireki.net/entertainers/3192


In [20]:
# トリオ詳細ページへのURLリストを作成

trio_page_urls = []

for combi_url in trio_urls:
    request = requests.get(combi_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    combi_pages = soup.select("#top > div > div > div > table > tbody > tr")
    for combi_page in combi_pages:
        combi_page_a = combi_page.select_one("td:nth-child(1) > a")
        url = combi_page_a.get("href")
        combi_name = combi_page_a.get_text(strip=True)
        trio_page_urls.append({"unit_name": combi_name, "page_url": url})
    time.sleep(1)

trio_page_df = pd.DataFrame(trio_page_urls)
trio_page_df

,unit_name,page_url
0,ビックスモールン,https://www.geireki.net/entertainers/964
1,リンダカラー,https://www.geireki.net/entertainers/1277
2,青色1号,https://www.geireki.net/entertainers/1702
3,AKIKO,https://www.geireki.net/entertainers/1703
4,あしゅら,https://www.geireki.net/entertainers/1704
...,...,...
160,大阪男塾,https://www.geireki.net/entertainers/3124
161,ぎょねこ,https://www.geireki.net/entertainers/3134
162,さんさんず,https://www.geireki.net/entertainers/3157
163,アマルフィん,https://www.geireki.net/entertainers/3162


In [30]:
# グループ詳細ページへのURLリストを作成

group_page_urls = []

for combi_url in [group_url_root]:
    request = requests.get(combi_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    combi_pages = soup.select("#top > div > div > div > table > tbody > tr")
    for combi_page in combi_pages:
        combi_page_a = combi_page.select_one("td:nth-child(1) > a")
        url = combi_page_a.get("href")
        combi_name = combi_page_a.get_text(strip=True)
        group_page_urls.append({"unit_name": combi_name, "page_url": url})
    time.sleep(1)

group_page_df = pd.DataFrame(group_page_urls)
group_page_df

,unit_name,page_url
0,ぼる塾,https://www.geireki.net/entertainers/2970
1,ザ・ドリフターズ,https://www.geireki.net/entertainers/2971
2,ザ・プラン９,https://www.geireki.net/entertainers/2986
3,超新塾,https://www.geireki.net/entertainers/3006
4,ショウガールズ,https://www.geireki.net/entertainers/3137
5,ダウ９００００,https://www.geireki.net/entertainers/3139


In [31]:
combi_page_df["unit"] = "combi"
pin_page_df["unit"] = "pin"
trio_page_df["unit"] = "trio"
group_page_df["unit"] = "group"

unit_page_df = pd.concat([combi_page_df, pin_page_df, trio_page_df, group_page_df]).reset_index(drop=True)
unit_page_df

,unit_name,page_url,unit
0,アームストロング,https://www.geireki.net/entertainers/1,combi
1,アイスクリーム,https://www.geireki.net/entertainers/2,combi
2,アイデンティティ,https://www.geireki.net/entertainers/3,combi
3,アイナラット,https://www.geireki.net/entertainers/4,combi
4,アイロンヘッド,https://www.geireki.net/entertainers/5,combi
...,...,...,...
3184,ザ・ドリフターズ,https://www.geireki.net/entertainers/2971,group
3185,ザ・プラン９,https://www.geireki.net/entertainers/2986,group
3186,超新塾,https://www.geireki.net/entertainers/3006,group
3187,ショウガールズ,https://www.geireki.net/entertainers/3137,group


In [32]:
unit_page_df.to_csv(os.path.join(folder_name_raw, "unit_page.csv"), index=False)

In [ ]:
# kokokara saikai

In [4]:
unit_page_df = pd.read_csv(os.path.join(folder_name_raw, "unit_page.csv"))
unit_page_df

,unit_name,page_url,unit
0,アームストロング,https://www.geireki.net/entertainers/1,combi
1,アイスクリーム,https://www.geireki.net/entertainers/2,combi
2,アイデンティティ,https://www.geireki.net/entertainers/3,combi
3,アイナラット,https://www.geireki.net/entertainers/4,combi
4,アイロンヘッド,https://www.geireki.net/entertainers/5,combi
...,...,...,...
3184,ザ・ドリフターズ,https://www.geireki.net/entertainers/2971,group
3185,ザ・プラン９,https://www.geireki.net/entertainers/2986,group
3186,超新塾,https://www.geireki.net/entertainers/3006,group
3187,ショウガールズ,https://www.geireki.net/entertainers/3137,group


In [6]:
# ユニットページから芸人ページURLを取得

performer_page_urls = []

for i, row in unit_page_df.iterrows():
    unit_name = row["unit_name"]
    print(unit_name)
    page_url = row["page_url"]
    unit = row["unit"]
    request = requests.get(page_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    tbody = soup.select_one("#top > div > table")
    if tbody:
        member_td = tbody.find("th", string="メンバー").find_next_sibling()
        members = member_td.select("tbody > tr")
        for member in members:
            performer = member.select_one("td:nth-child(1) > a")
            performer_url = performer.get("href")
            performer_name = performer.get_text(strip=True)
            performer_page_urls.append({"unit_name": unit_name, "unit_url": page_url, "unit": unit, "performer_name": performer_name, "performer_url": performer_url})
    else:
        performer_page_urls.append({"unit_name": unit_name, "unit_url": page_url, "unit": unit, "performer_name": None, "performer_url": None})
    time.sleep(1)

performer_page_df = pd.DataFrame(performer_page_urls)
performer_page_df


アームストロング
アイスクリーム
アイデンティティ
アイナラット
アイロンヘッド
アインシュタイン
青芝金太・紋太
青芝フック・キック
青空一風・千風
青空一歩・三歩
青空球児・好児
青空千夜・一夜
青空東児・西児
あかいらか
あがすけ
暁照雄・光雄
あかぼし☆こぼし
赤もみじ
アキカン
あきげん
秋田一号・二号
秋田Aスケ・Bスケ
アキナ
秋山右楽・左楽
アクシャン
あくび虫
アゲイン
あご&きんぞう
浅草キッド
朝倉小松崎
あさり・かつお
あさりど
アジアントール
足軽エンペラー
東京二・京太
東京丸・京平
東ヤジロー・キタハチ
遊び屋
アッチャンズ
アッパレード
アップダウン
あどばるーん
ADVANCE
アナクロニスティック
アニマル梯団
あばれヌンチャク
アビリティモンキー
あひるちゃんスポンジクラブ
アホマイルド
あま福
アマレス兄弟
アミ・ボロロク・ホエ・デシェ・ジェテ・チャイ
雨上がり決死隊
アメデオ
アメリカザリガニ
アメリカンコミックス
あめん
アモーン
荒波部屋
ありがとう
アリtoキリギリス
ありんくりん
アルカリ三世
アルコ&ピース
アルファルファ
あわよくば
アンガールズ
アンカンミンカン
アンクルベイビー
アンジャッシュ
ANZEN漫才
アンダーエイジ
アンダーポイント
アンダーライン
アンタッチャブル
アントニオ
アンドロイド
アントワネット
アンバランス
あんぺあ
イアソン
いい塩梅
EE男
Yes-man
庵。
いかすぜジョナサン
粋なり
EXIT
いけないパラダイス
囲碁将棋
いきるゆうき
イシバシハザマ
いじわるコンビ
イタガキ
イタリアン・シガー・ブルドッグ
市原
いち・もく・さん
一輪亭花蝶・松原勝美
いつもここから
イナズマライン
いぬ
イヌがニャーと泣いた日
イヌコネクション
犬の心
井下好井
伊村製作所
入間国際宣言
イワイガワ
インディアンス
インディゴ
インデペンデンスデイ
インパルス
インポッシブル
ヴィレッジ
ういろうプリン
ヴィンテージ
ウーマンラッシュアワー
烏龍パーク
ヴェートーベン
上木総合研究所
ウエストランド
うえはまだ
ウェルダン
ウォーターズ
浮世亭出羽助・八丈竹幸
浮世亭柳平・とん平
うしろシティ
うすくら屋
うずしおパンチ
ウッチャンナンチャン
ウドントミカン
海原はるか・かなた
ウメボシ

,unit_name,unit_url,unit,performer_name,performer_url
0,アームストロング,https://www.geireki.net/entertainers/1,combi,くり,https://www.geireki.net/perfomers/703
1,アームストロング,https://www.geireki.net/entertainers/1,combi,とにかく明るい安村,https://www.geireki.net/perfomers/1379
2,アイスクリーム,https://www.geireki.net/entertainers/2,combi,ムーディ勝山,https://www.geireki.net/perfomers/2066
3,アイデンティティ,https://www.geireki.net/entertainers/3,combi,見浦彰彦,https://www.geireki.net/perfomers/1990
4,アイデンティティ,https://www.geireki.net/entertainers/3,combi,田島直弥,https://www.geireki.net/perfomers/1211
...,...,...,...,...,...
2614,ダウ９００００,https://www.geireki.net/entertainers/3139,group,道上 珠妃,https://www.geireki.net/perfomers/3007
2615,ダウ９００００,https://www.geireki.net/entertainers/3139,group,上原 佑太,https://www.geireki.net/perfomers/3008
2616,ダウ９００００,https://www.geireki.net/entertainers/3139,group,忽那 文香,https://www.geireki.net/perfomers/3009
2617,ダウ９００００,https://www.geireki.net/entertainers/3139,group,中島 百依子,https://www.geireki.net/perfomers/3010


In [7]:
performer_page_df.to_csv(os.path.join(folder_name_raw, "performer_page.csv"), index=False)

In [ ]:
# 芸人ページから、誕生日・年齢・出身地・事務所・芸歴を取得

performer_data = []

for i, row in performer_page_df.iterrows():
    if i==1: break
    unit_name = row["unit_name"]
    unit_url = row["unit_url"]
    unit = row["unit"]
    performer_name = row["performer_name"]
    print(i, performer_name)
    performer_url = row["performer_url"]
    request = requests.get(performer_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    tbody = soup.select_one("#top > div > table")
    birthday = tbody.find("th", string="誕生日").find_next_sibling().get_text(strip=True)
    age = tbody.find("th", string="年齢").find_next_sibling().get_text(strip=True)
    birthplace = tbody.find("th", string="出身地").find_next_sibling().get_text(strip=True)
    office = tbody.find("th", string="事務所").find_next_sibling().get_text(strip=True)
    history = tbody.find("th", string="芸歴").find_next_sibling().get_text(strip=True)
    performer_data.append({"unit_name": unit_name, "unit_url": unit_url, "unit": unit, "performer_name": performer_name, "performer_url": performer_url,
                           "birthday": birthday, "age": age, "birthplace": birthplace, "office": office, "history": history})
    time.sleep(1)
    

performer_data_df = pd.DataFrame(performer_data)
performer_data_df


In [ ]:
performer_data_df.to_csv(os.path.join(folder_name_raw, "performer_data.csv"), index=False)
